In [1]:
import random
import time
from copy import deepcopy

class TicTacDoh:
    def __init__(self):
        self.board = [" "] * 9
        self.current_player = "X"

    def display(self):
        for i in range(3):
            print("|".join(self.board[i*3:(i+1)*3]))
            if i < 2:
                print("-"*5)

    def possible_moves(self):
        return [i for i, cell in enumerate(self.board) if cell == " "]

    def make_move(self, move):
        self.board[move] = self.current_player

    def switch_player(self):
        self.current_player = "O" if self.current_player == "X" else "X"

    def game_over(self):
        return self.winner() is not None or " " not in self.board

    def winner(self):
        wins = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],
            [0, 3, 6], [1, 4, 7], [2, 5, 8],
            [0, 4, 8], [2, 4, 6]
        ]
        for combo in wins:
            a, b, c = combo
            if self.board[a] == self.board[b] == self.board[c] != " ":
                return self.board[a]
        return None

    def clone(self):
        new_game = TicTacDoh()
        new_game.board = self.board[:]
        new_game.current_player = self.current_player
        return new_game

    def score(self):
        winner = self.winner()
        if winner == "X":
            return 1
        elif winner == "O":
            return -1
        else:
            return 0

def negamax(game, depth, player):
    if depth == 0 or game.game_over():
        return player * game.score(), None

    best_value = float('-inf')
    best_move = None
    for move in game.possible_moves():
        new_game = game.clone()
        new_game.make_move(move)
        new_game.switch_player()
        val, _ = negamax(new_game, depth - 1, -player)
        val = -val
        if val > best_value:
            best_value = val
            best_move = move
    return best_value, best_move

def negamax_ab(game, depth, alpha, beta, player):
    if depth == 0 or game.game_over():
        return player * game.score(), None

    best_value = float('-inf')
    best_move = None
    for move in game.possible_moves():
        new_game = game.clone()
        new_game.make_move(move)
        new_game.switch_player()
        val, _ = negamax_ab(new_game, depth - 1, -beta, -alpha, -player)
        val = -val
        if val > best_value:
            best_value = val
            best_move = move
        alpha = max(alpha, val)
        if alpha >= beta:
            break
    return best_value, best_move

def run_comparison(games_count=1000):
    algorithms = {
        "Negamax": lambda g, d, p: negamax(g, d, p),
        "NegamaxAB": lambda g, d, p: negamax_ab(g, d, float('-inf'), float('inf'), p)
    }
    for algo_name, algo_fn in algorithms.items():
        print(f"\nAlgorytm: {algo_name}")
        for probabilistic in [False, True]:
            print(f"Wariant {'probabilistyczny' if probabilistic else 'deterministyczny'}:")
            for depth in [2, 4]:
                results = {"X": 0, "O": 0, "Draw": 0}
                total_time = 0
                total_moves = 0
                for i in range(games_count):
                    game = TicTacDoh()
                    game.current_player = "X" if i % 2 == 0 else "O"
                    while not game.game_over():
                        start = time.perf_counter()
                        _, move = algo_fn(game, depth, 1 if game.current_player == "X" else -1)
                        total_time += time.perf_counter() - start
                        total_moves += 1

                        if move is not None:
                            if probabilistic and random.random() < 0.2:
                                # Ruch nieudany – przeciwnik przechodzi do ruchu
                                game.switch_player()
                                continue

                            game.make_move(move)

                            if not game.game_over():
                                game.switch_player()
                        else:
                            break
                    winner = game.winner()
                    if winner:
                        results[winner] += 1
                    else:
                        results["Draw"] += 1
                avg_time = total_time / total_moves if total_moves > 0 else 0
                print(f"  Głębokość {depth} → X: {results['X']} | O: {results['O']} | Remisy: {results['Draw']} | Średni czas decyzji: {avg_time:.6f} s")

if __name__ == "__main__":
    run_comparison()



Algorytm: Negamax
Wariant deterministyczny:
  Głębokość 2 → X: 500 | O: 500 | Remisy: 0 | Średni czas decyzji: 0.000074 s
  Głębokość 4 → X: 0 | O: 0 | Remisy: 1000 | Średni czas decyzji: 0.001387 s
Wariant probabilistyczny:
  Głębokość 2 → X: 481 | O: 468 | Remisy: 51 | Średni czas decyzji: 0.000073 s
  Głębokość 4 → X: 327 | O: 346 | Remisy: 327 | Średni czas decyzji: 0.001574 s

Algorytm: NegamaxAB
Wariant deterministyczny:
  Głębokość 2 → X: 500 | O: 500 | Remisy: 0 | Średni czas decyzji: 0.000036 s
  Głębokość 4 → X: 0 | O: 0 | Remisy: 1000 | Średni czas decyzji: 0.000202 s
Wariant probabilistyczny:
  Głębokość 2 → X: 470 | O: 498 | Remisy: 32 | Średni czas decyzji: 0.000036 s
  Głębokość 4 → X: 338 | O: 358 | Remisy: 304 | Średni czas decyzji: 0.000222 s
